# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://www.telstra.com.au/")
ed.links

['#main-content',
 'https://www.telstra.com.au',
 'https://www.telstra.com.au',
 'https://www.telstra.com.au',
 'https://www.telstra.com.au',
 '/mobile-phones/prepaid-mobiles/49-prepaid-sim-kit',
 '/mobile-phones/prepaid-mobiles/offers-and-rates#sim-plans',
 'https://telstra.app.link/c8a3dd40',
 'https://telstra.app.link/f3ab3ea5',
 'https://fix.telstra.com/',
 '/mobile-phones/activate',
 '/support',
 'https://plus.telstra.com.au',
 '#lightbox-tnc',
 '/internet/plans',
 '/internet/plans',
 '/mobile-phones',
 '/mobile-phones/mobiles-on-a-plan',
 '/mobile-phones/sim-only-plans',
 '/mobile-phones/prepaid-mobiles',
 '/mobile-phones',
 '/internet',
 '/internet/plans',
 '/entertainment',
 '/entertainment/gaming',
 '/internet',
 '/accessories',
 '/accessories/watches-wearable',
 '/accessories/headphones-speakers',
 '/tablets/tablets-on-a-plan',
 '/accessories',
 'https://plus.telstra.com.au',
 'https://plus.telstra.com.au/rewards',
 'https://plus.telstra.com.au/tickets',
 'https://www.telstra

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.telstra.com.au/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main-content
https://www.telstra.com.au
https://www.telstra.com.au
https://www.telstra.com.au
https://www.telstra.com.au
/mobile-phones/prepaid-mobiles/49-prepaid-sim-kit
/mobile-phones/prepaid-mobiles/offers-and-rates#sim-plans
https://telstra.app.link/c8a3dd40
https://telstra.app.link/f3ab3ea5
https://fix.telstra.com/
/mobile-phones/activate
/support
https://plus.telstra.com.au
#lightbox-tnc
/internet/plans
/internet/plans
/mobile-phones
/mobile-phones/mobiles-on-a-plan
/mobile-phones/sim-only-plans
/mobile-phones/prepaid-mobiles
/mobile-phones
/internet
/internet/plans
/entertainment
/entertainment/gaming
/internet
/accessories
/accessories/watches-wearable
/accessories/headphones-speak

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/ACE-Step/ACE-Step-v1-3.5B',
 '/nari-labs/Dia-1.6B',
 '/Lightricks/LTX-Video',
 '/lodestones/Chroma',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/smolagents/computer-agent',
 '/spaces/ByteDance/DreamO',
 '/spaces/ACE-Step/ACE-Step',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/DMindAI/DMind_Benchmark',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/d

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nvidia/parakeet-tdt-0.6b-v2
Updated
14 days ago
•
148k
•
833
ACE-Step/ACE-Step-v1-3.5B
Updated
2 days ago
•
410
nari-labs/Dia-1.6B
Updated
1 day ago
•
164k
•
2.15k
Lightricks/LTX-Video
Updated
about 4 hours ago
•
275k
•
1.47k
lodestones/Chroma
Updated
1 day ago
•
516
Browse 1M+ models
Spaces
Running
6.54k
6.54k
DeepSite
🐳
Generate any application with DeepSeek
Running
on

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("Telstra", "https://www.telstra.com.au/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.telstra.com.au/aboutus'}, {'type': 'contact page', 'url': 'https://www.telstra.com.au/contact-us'}]}


"You are looking at a company called: Telstra\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nTelstra Personal\nWebpage Contents:\nPublisher:\nTelstra\nName:\nTelstra Personal\nCopyrighted By:\nTelstra\nFamily Friendly:\nYes\nLanguage:\nEnglish\nSkip to main content\nTelstra\nTelstra\nTelstra\nPersonal\nBusiness\nEnterprise\nTelstra Plus\nSign in\nTelstra\nTelstra Personal\nTry Pre-Paid on Australia’s most reliable network\nEnjoy unlimited calls and texts to standard numbers within Australia on our $49 Pre-Paid Plan and save $32 while you’re at it. Offer ends 2 June 2025.\nNew activations only. Future recharges will be full price. For use in Aus.\nExplore deal\nExplore Pre-Paid\nMake a payment\nRecharge Pre-Paid\nFix an issue\nActivate my mobile\nGet help\nTelstra Plus\nGet up to $25/mth off nbn for 6 months\nOn selected nbn® home internet plans. Smart Modem i

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Telstra", "https://www.telstra.com.au/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.telstra.com.au/aboutus'}, {'type': 'careers page', 'url': 'https://www.telstra.com.au/careers'}, {'type': 'contact page', 'url': 'https://www.telstra.com.au/contact-us'}]}


# **Telstra Company Brochure**

---

## **About Telstra**

Telstra is Australia's leading telecommunications and technology company. With a strong commitment to building a sustainable and connected future, we ensure that our services meet the needs of individuals, families, and businesses across the nation. Our extensive mobile network covers around 1 million square kilometers more than any other network in Australia, making us the best option for reliable connectivity.

---

## **Our Services**

### **Telstra Personal**
- **Mobile Plans**: Explore our Pre-Paid plans with unlimited calls and texts to standard numbers within Australia.
- **Home Internet**: Get faster home internet with our NBN plans, offering discounted rates for new services.
- **Accessories**: A wide selection of the latest mobile phones, tablets, and wearable tech.

### **Telstra Plus**
Join Telstra Plus to earn rewards! Enjoy benefits such as discounted movie tickets and exclusive offers. 

---

## **Customer Commitment**

At Telstra, we prioritize our customers' needs. We recognize the significance of community and family, supporting them through various initiatives including:
- **Payment Assistance**: Flexible options to help keep you connected during tough times.
- **Support for Aboriginal Communities**: Our respect for First Nations people reflects in our commitment to protect their digital presence.

---

## **Company Culture**

Telstra is more than just a telecommunications company; we are a community that values respect, inclusivity, and sustainability. We aim to build a prosperous Australia by fostering strong connections not just among our customers but also with our communities. This commitment extends to acknowledging the traditional custodians of the land and prioritizing positive impacts on the environment.

---

## **Careers at Telstra**

Joining Telstra means becoming part of an innovative and inclusive team. We are always looking for talented individuals who are passionate about technology and customer service. Whether you are entering the workforce or looking for your next opportunity, Telstra offers pathways for careers that can make a real difference.

### **Discover Your Future**
- **Innovative Tech Roles**: Work in a company that embraces the latest technology.
- **Customer-Centric Positions**: Find a role where you can directly impact our customers' experience.

---

## **Join Us!**

Whether you're a prospective customer, investor or looking for a career, Telstra invites you to explore the possibilities. Embrace reliable connectivity, rewarding careers, and a sustainable future with us.

**Contact Us**: Visit our [website](https://www.telstra.com.au) for more details, or download the My Telstra app for exclusive offers and features.

--- 
*Telstra is committed to building a connected and inclusive Australia for everyone.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Telstra", "https://www.telstra.com.au/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.telstra.com.au/aboutus/support-in-times-of-need/adversity-financial-hardship'}, {'type': 'contact page', 'url': 'https://www.telstra.com.au/contact-us'}]}


# Telstra Company Brochure

Welcome to Telstra, Australia's leading telecommunications and technology company. Committed to connecting Australians and empowering communities through innovative solutions, we serve a diverse range of customers, from individual consumers to large enterprises.

## Company Overview

**Founded**: Established to provide reliable telecommunications across Australia, Telstra has grown into a trusted name in connectivity solutions. 

**Mission**: At Telstra, our aim is to connect the country and deliver exceptional services that enhance the everyday lives of our customers.

## Our Products and Services

- **Mobile Connectivity**: Experience Australia's most reliable mobile network with our extensive range of mobile plans, including **Pre-Paid** options offering unlimited calls and texts to standard numbers within Australia at competitive prices.
  
- **Internet Solutions**: Offering some of the fastest home internet plans available, including **NBN** options, and the revolutionary **Smart Modem**, designed to keep you connected effortlessly.

- **Accessories and Gadgets**: Discover a wide variety of accessories, including the latest mobile devices, tablets, and wearables.

- **Rewards Program**: Join **Telstra Plus** to earn points for every purchase, offering you discounts on outstanding brands and products.

## Company Culture

At Telstra, we acknowledge the importance of fostering an inclusive workplace that values diversity and supports our employees. We believe in championing Indigenous communities, respecting their connections to the land, and working together to build an inclusive tomorrow. Our commitment extends to providing assistance in challenging times, ensuring financial relief options are available to help our customers stay connected.

## Customer Commitment

Telstra prioritizes customer satisfaction by providing exceptional service and support. Whether through our **My Telstra app** for easy account management or our dedicated customer service teams, we offer help and support tailored to individual needs. 

### Payment Assistance
To support customers facing financial hardship, Telstra provides flexible options that ensure no one has to face communication barriers during tough times.

## Career Opportunities

Telstra aspires to be an employer of choice, offering dynamic career paths for talented professionals. We are constantly on the lookout for passionate individuals who are eager to contribute to our mission. Potential team members can expect a supportive environment that encourages personal growth, learning, and development.

### Why Join Us?
- Work with cutting-edge technology that shapes the future of connectivity.
- Be part of a community-driven organization committed to making a difference.
- Access flexible career options that work for you, whether you're in tech, customer service, or business development.

---

Join us on the journey towards a more connected and inclusive Australia! For more information, visit our website or check out our career opportunities today.

**Contact Us**  
Website: [Telstra](https://www.telstra.com.au)  
Follow us on social media to get the latest updates!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Telstra", "https://www.telstra.com.au/")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>